# Parsing

## Starting things up

Run:

    make start-dev-containers

Now, let's export some environment variables we need:

In [1]:
export USER_CERT=../../tests/certs/test_user_curl.pem
export BASE_URL=https://ntc-rosetta-conf:8443

## Configuration

First we need to configure the platform of the device, this configuration is part of the model.

In [2]:
cat 5_parse/configuration.json

{
    "ntc-rosetta-conf:device": {
        "config": {
            "platform": "ios"
        }
    }
}


In [3]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @5_parse/configuration.json \
    $BASE_URL/restconf/data/

In [4]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "status": "OK",
    "conf-changed": true
}

In [5]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf_running/data/ntc-rosetta-conf:device

{
    "ntc-rosetta-conf:device": {
        "config": {
            "platform": "ntc-rosetta-conf:ios"
        }
    }
}

## Parsing

Now let's see how we can parse native configuration, let's start by checking configuration is empty:

In [6]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
     $BASE_URL/restconf_running/data/openconfig-interfaces:interfaces 

{
    "openconfig-interfaces:interfaces": {
        "interface": []
    }
}

Now we need to a json object with the following structure:

```
{
    "ntc-rosetta-conf:input": {
        "validate": true,
        "native": "string with configuration in native format (ios_style/xml/etc)"
    }
}
```

In [7]:
cat 5_parse/parse.json

{
    "ntc-rosetta-conf:input": {
        "validate": true,
        "native": "interface GigabitEthernet0\n   description an interface description\n   exit\n!\ninterface GigabitEthernet1\n   description another interface\n   exit\n!\n"
    }
}


Now we call the RPC `ntc-rosetta-conf:parse`:

In [8]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @5_parse/parse.json \
    $BASE_URL/restconf/operations/ntc-rosetta-conf:parse

{
    "result": "ntc-rosetta-config:success"
}

Configuration is parsed and loaded into the running database:

In [9]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf_running/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "GigabitEthernet0",
                "config": {
                    "name": "GigabitEthernet0",
                    "type": "iana-if-type:ethernetCsmacd",
                    "description": "an interface description",
                    "enabled": true
                }
            },
            {
                "name": "GigabitEthernet1",
                "config": {
                    "name": "GigabitEthernet1",
                    "type": "iana-if-type:ethernetCsmacd",
                    "description": "another interface",
                    "enabled": true
                }
            }
        ]
    }
}

In [10]:
# ignore me, this deletes the data so the notebook can be rerun
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X PUT \
    -d @../../tests/data/interfaces_empty.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X DELETE \
    $BASE_URL/restconf/data/ntc-rosetta-conf:device
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "status": "OK",
    "conf-changed": true
}